In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
df = pd.read_excel("Transaction Data.xlsx", sheet_name=None)
data = df["Data"]

closed_position = df["Closed Position"]
open_position = df["Open Position"]

In [ ]:
# filter
date_options = [{"label":d, "value":d} for d in closed_position.sort_values("Date_Close", ascending=False)["Date"].unique()]
date_options.insert(0,{"label":"All","value":"All"})
date_options

In [ ]:
type_options =  [{"label":d, "value":d} for d in closed_position["Type"].unique()]
type_options

In [ ]:
# main kpi
total_pl = closed_position["P/L (SGD)"].sum()
total_pl

In [ ]:
# line
closed_position["Date"] = closed_position["Date_Close"].dt.strftime("%b-%y")

cul_pl = closed_position.groupby("Date").sum()[["P/L (SGD)"]].rename({"P/L (SGD)":"P/L"}, axis=1).reset_index()
cul_pl["DATE(SORT)"] = pd.to_datetime(cul_pl["Date"], format="%b-%y")
cul_pl = cul_pl.sort_values(["DATE(SORT)"]).drop("DATE(SORT)", axis=1)
cul_pl["Cumulative P/L"] = cul_pl["P/L"].cumsum()
cul_pl["Cumulative P/L"] = cul_pl["Cumulative P/L"].map(lambda x: round(x, 2))
cul_pl

In [ ]:
# bar
pl_by_type = closed_position.groupby("Type")[["P/L (SGD)"]].sum().reset_index()

pl_by_type["P/L"] = pl_by_type["P/L (SGD)"].map(lambda x: round(x,2))
pl_by_type["TEXT"] = pl_by_type["P/L"].map(lambda x:  "-$" + str(abs(x)) if x<0 else "$" + str((x)))
pl_by_type = pl_by_type.sort_values(["P/L"])
pl_by_type

In [ ]:
# stacked bar
data["DATE"] = data["Date"].dt.strftime("%b-%y")

trade = data.groupby(["DATE","Action"])[["Symbol"]].count().rename({"Symbol":"No. Trades"}, axis=1).reset_index()
trade["DATE(SORT)"] = pd.to_datetime(trade["DATE"], format="%b-%y")
trade = trade.sort_values(["DATE(SORT)"]).drop("DATE(SORT)", axis=1)
trade

In [ ]:
# P/L by name
pl_by_name = closed_position.groupby(["Type","Name"])[["P/L (SGD)","P/L (%)"]].sum().rename({"P/L (SGD)":"P/L"}, axis=1).reset_index()
pl_by_name

### Provide Fixtures for Django Model

In [3]:
import json
from datetime import date

In [ ]:
# tickerinfo
tickerinfo = data[["Symbol","Name","Type","Currency","Sector","Industry"]].drop_duplicates().rename({"Symbol":"pk"},axis=1).copy()
tickerinfo["model"] = "api.tickerinfo"
tickerinfo.columns = tickerinfo.columns.str.lower()

In [ ]:
fixture = []

for row in tickerinfo.iterrows():
    _ = {"model":row[1]["model"],
    "pk":row[1]["pk"],
    "fields":
        row[1][["name","type","currency","sector","industry"]].to_dict()
    }
    
    fixture.append(_)
    
with open("tickerinfo.json","w") as f:
    json.dump(fixture, f)

In [4]:
# transactionmodel
transactionmodel = data[["Date","Symbol","Action","Price","Shares","Comm","Exchange Rate","Amount","Amount (SGD)"]].rename({"Shares":"Quantity",
                                                                                                                           "Comm":"Fees",
                                                                                                                           "Amount":"Value",
                                                                                                                           "Amount (SGD)":"Value_sgd",
                                                                                                                           "Exchange Rate":"exchange_rate"}, axis=1).copy()
transactionmodel.columns = transactionmodel.columns.str.lower()
transactionmodel["date"] = pd.to_datetime(transactionmodel["date"], format="%Y-%m-%d")
transactionmodel["date"] = transactionmodel["date"].map(lambda x: date(x.year, x.month,x.day).__str__())
transactionmodel["model"] = "api.transactionmodel"
transactionmodel["pk"] = transactionmodel["symbol"] + "|" + transactionmodel["date"].astype(str) + "|" + transactionmodel["action"]

In [5]:
fixture = []

for row in transactionmodel.iterrows():
    _ = {"model":row[1]["model"],
    "pk":row[1]["pk"],
    "fields":
        row[1][[col for col in transactionmodel.columns if col not in ["model","pk"]]].to_dict()
    }
    
    fixture.append(_)
    
with open("transactionmodel.json","w") as f:
    json.dump(fixture, f)

In [21]:
# closed position
closedposition = closed_position[["Symbol","Date_Open","Date_Close","Holding (days)","P/L (Origin Currency)","P/L (SGD)","P/L (%)"]].rename({"Holding (days)":"Holding",
                                                                                                                      "P/L (Origin Currency)":"pl",
                                                                                                                      "P/L (SGD)":"pl_sgd",
                                                                                                                      "P/L (%)":"pl_per"}, axis=1).copy()
closedposition.columns = closedposition.columns.str.lower()
for dat in ["date_open","date_close"]:
    closedposition[dat] = pd.to_datetime(closedposition[dat], format="%Y-%m-%d")
    closedposition[dat] = closedposition[dat].map(lambda x: date(x.year, x.month,x.day).__str__())
closedposition["model"] = "api.closedposition"
closedposition["pk"] = closedposition["symbol"] + "|" + closedposition["date_open"].astype(str) + "|" + closedposition["date_close"].astype(str)

In [22]:
fixture = []

for row in closedposition.iterrows():
    _ = {"model":row[1]["model"],
    "pk":row[1]["pk"],
    "fields":
        row[1][[col for col in closedposition.columns if col not in ["model","pk"]]].to_dict()
    }
    
    fixture.append(_)
    
with open("closedposition.json","w") as f:
    json.dump(fixture, f)

In [30]:
# openposition
openposition = open_position[["Date","Symbol","Avg Price","Shares","Comm","Exchange Rate","Amount","Amount (SGD)","Holdings (days)","Weightage","P/L (Origin Currency)","P/L (SGD)","P/L (%)","Value (SGD)"]].rename({"Date":"Date_open","Avg Price":"avg_price","Shares":"total_quantity","Comm":"total_fees","Exchange Rate":"avg_exchange_rate","Amount":"total_value","Amount (SGD)":"total_value_sgd","Holdings (days)":"total_holding","P/L (Origin Currency)":"unrealised_pl","P/L (SGD)":"unrealised_pl_sgd","P/L (%)":"unrealised_pl_per","Value (SGD)":"current_value_sgd"}, axis=1)
openposition.columns = openposition.columns.str.lower()
openposition["model"] = "api.openposition"
openposition["date_open"] = pd.to_datetime(openposition["date_open"], format="%Y-%m-%d")
openposition["date_open"] = openposition["date_open"].map(lambda x: date(x.year, x.month,x.day).__str__())
openposition["pk"] = openposition["symbol"] + "|" + openposition["date_open"]

In [32]:
fixture = []

for row in openposition.iterrows():
    _ = {"model":row[1]["model"],
    "pk":row[1]["pk"],
    "fields":
        row[1][[col for col in openposition.columns if col not in ["model","pk"]]].to_dict()
    }
    
    fixture.append(_)
    
with open("openposition.json","w") as f:
    json.dump(fixture, f)

### Test request

In [5]:
import requests

data = {"date":"2021-08-10",
       "symbol":"TSLA",
       "action":"Buy",
       "price":1000,
       "quantity":1,
       "fees":1,
       "exchange_rate":1.3}

response = requests.post("http://127.0.0.1:8000/api/transaction", data=data)

In [8]:
response.status_code == 200

True